In [1]:
import numpy as np
import pandas as pd
import cv2
import os

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
TrainImagePaths = []
for dirname, _, filenames in os.walk('/kaggle/input/shopee-product-detection-student/train/train/train'):
    for filename in filenames:
        if (filename[-3:] == 'jpg'):
            TrainImagePaths.append(os.path.join(dirname, filename))

In [4]:
imgSize = 50
X_train = []
Y_train = []
for imagePath in tqdm(TrainImagePaths):
    label = imagePath.split(os.path.sep)[-2]
#     image = cv2.imread(imagePath)
    image = cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (imgSize, imgSize))

    X_train.append(image)
    Y_train.append(int(label))
    
X_train = np.array(X_train).astype('float16')/255
Y_train = to_categorical(Y_train)

100%|██████████| 105390/105390 [10:23<00:00, 168.96it/s]


In [74]:
to_categorical(int(imagePath.split(os.path.sep)[-2]))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1.], dtype=float32)

In [5]:
X_train = X_train.reshape(105390,imgSize,imgSize,1)

X_train.shape

(105390, 50, 50, 1)

In [6]:
model = Sequential()
model.add(Conv2D(42, kernel_size=(3, 3),activation='relu',input_shape=(imgSize,imgSize,1)))
model.add(Flatten())
model.add(Dense(42, activation='softmax'))

In [7]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=15)

Epoch 1/15
3294/3294 [==============================] - 117s 36ms/step - loss: 3.0996 - accuracy: 0.2136
Epoch 2/15
3294/3294 [==============================] - 116s 35ms/step - loss: 2.2932 - accuracy: 0.4034
Epoch 3/15
3294/3294 [==============================] - 114s 35ms/step - loss: 1.5351 - accuracy: 0.5961
Epoch 4/15
3294/3294 [==============================] - 115s 35ms/step - loss: 0.9571 - accuracy: 0.7524
Epoch 5/15
3294/3294 [==============================] - 113s 34ms/step - loss: 0.5966 - accuracy: 0.8524
Epoch 6/15
3294/3294 [==============================] - 112s 34ms/step - loss: 0.3904 - accuracy: 0.9087
Epoch 7/15
3294/3294 [==============================] - 113s 34ms/step - loss: 0.2746 - accuracy: 0.9397
Epoch 8/15
3294/3294 [==============================] - 114s 35ms/step - loss: 0.2040 - accuracy: 0.9576
Epoch 9/15
3294/3294 [==============================] - 114s 34ms/step - loss: 0.1588 - accuracy: 0.9694
Epoch 10/15
3294/3294 [==============================] 

In [83]:
df = pd.read_csv('../input/shopee-product-detection-student/test.csv')
X_test = []
for imageName in tqdm(df['filename']): 
    image = cv2.imread('../input/shopee-product-detection-student/test/test/test/'+imageName, cv2.IMREAD_GRAYSCALE)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (imgSize, imgSize))
    X_test.append(image)
X_test = np.array(X_test).astype('float16')/255

100%|██████████| 12186/12186 [00:51<00:00, 237.85it/s]


In [84]:
X_test = X_test.reshape(12186,imgSize,imgSize,1)

X_test.shape

(12186, 50, 50, 1)

In [85]:
res = model.predict(X_test, batch_size=32)
res = np.argmax(res, axis=1)
# print(res)
df['category'] = res
df['category'] = df.category.apply(lambda c: str(c).zfill(2))
df.to_csv('output1-7-3.csv', index = False)

COBA TRAIN

In [29]:
df

,filename,category
0,fd663cf2b6e1d7b02938c6aaae0a32d2.jpg,12
1,c7fd77508a8c355eaab0d4e10efd6b15.jpg,27
2,127f3e6d6e3491b2459812353f33a913.jpg,38
3,5ca4f2da11eda083064e6c36f37eeb81.jpg,16
4,46d681a542f2c71be017eef6aae23313.jpg,39
...,...,...
12181,5ba958eacb23cd7d1673bad4dae55784.jpg,10
12182,efbe41a1c2b666b70e337e438559808b.jpg,19
12183,79fdaa5ac5ba10dbe8004cabd8c35eb3.jpg,34
12184,ac3d136124617637a05ba66694e381ef.jpg,31


In [89]:
# df = pd.read_csv('../input/shopee-product-detection-student/test.csv')
# X_test = []
# for imageName in tqdm(df['filename'][0:10]): 
image = cv2.imread('../input/shopee-product-detection-student/test/test/test/004316305306b74ecd1c9a887954aa17.jpg', cv2.IMREAD_GRAYSCALE)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (imgSize, imgSize))

X_test=[]
X_test.append(image)
X_test = np.array(X_test).astype('float16')/255

X_test = X_test.reshape(1,imgSize,imgSize,1)
X_test.shape

res = model.predict(X_test, batch_size=32)
res = np.argmax(res, axis=1)

print(res)

# df['category'] = res
# df['category'] = df.category.apply(lambda c: str(c).zfill(2))
# df.to_csv('outputcobacoba.csv', index = False)

[38]


In [51]:
df['filename'][0:10]

0    fd663cf2b6e1d7b02938c6aaae0a32d2.jpg
1    c7fd77508a8c355eaab0d4e10efd6b15.jpg
2    127f3e6d6e3491b2459812353f33a913.jpg
3    5ca4f2da11eda083064e6c36f37eeb81.jpg
4    46d681a542f2c71be017eef6aae23313.jpg
5    f27ffaf1ab259a0efe2c59611de6732b.jpg
6    5eb8d9688d86ec7277f10fc18f386913.jpg
7    2ff578b587ec0c086ded573a3a9a9e3a.jpg
8    936b60783f4f3d3f2b718cc28a1cd629.jpg
9    2ac0233cc59040fd847f0cd9528772cc.jpg
Name: filename, dtype: object